In [1]:
# !pip install numpy==1.21.6s

In [2]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from scipy.spatial.distance import cdist, pdist, squareform, euclidean
import os
import pickle
from sklearn.model_selection import train_test_split
from collections import Counter
from tqdm import tqdm
import gensim
import ot

Customize the dataset

In [3]:
# existing_csv_path = './Dataset/Dunnhumby Dataset/transaction_data.csv'

# # Read the existing CSV file into a DataFrame
# df = pd.read_csv(existing_csv_path)

# column_mapping = {
#     'household_key': 'user_id',
#     'BASKET_ID': 'order_id',
#     'PRODUCT_ID': 'product_id',
#     'QUANTITY': 'quantity',
# }

# # Rename columns
# df.rename(columns=column_mapping, inplace=True)
# df.sort_values(by=['user_id', 'DAY'], inplace=True)
# df['days_since_prior_order'] = df.groupby('user_id')['DAY'].diff()
# df['days_since_prior_order'] = df['days_since_prior_order'].fillna(0).astype(int)
# new_csv_path = './Dataset/Dunnhumby Dataset/updated_data.csv'

# # Save the modified DataFrame to a new CSV file
# df.to_csv(new_csv_path, index=False)

In [4]:
path_train = "./Dataset/instacart-market-basket-analysis/order_products__train.csv"
path_prior = "./Dataset/instacart-market-basket-analysis/order_products__prior.csv"
path_products = "./Dataset/instacart-market-basket-analysis/products.csv"

train_orders = pd.read_csv(path_train)
prior_orders = pd.read_csv(path_prior)
products = pd.read_csv(path_products)

#Turn the product ID to a string
#This is necessary because Gensim's Word2Vec expects sentences, so we have to resort to this dirty workaround
train_orders["product_id"] = train_orders["product_id"].astype(str)
prior_orders["product_id"] = prior_orders["product_id"].astype(str)

train_products = train_orders.groupby("order_id").apply(lambda order: order['product_id'].tolist())
prior_products = prior_orders.groupby("order_id").apply(lambda order: order['product_id'].tolist())

#Create the final sentences
sentences = prior_products._append(train_products).values

#Train Word2Vec model
model = gensim.models.Word2Vec(sentences, vector_size=50, window=5, min_count=50, workers=4)

model.save("./product2vec.model")
model.wv.save_word2vec_format("./product2vec.model.bin", binary=True)

In [5]:
class BasketConstructor(object):
    '''
        Group products into baskets(type: list)
    '''
    def __init__(self, raw_data_dir, cache_dir):
        self.raw_data_dir = raw_data_dir
        self.cache_dir = cache_dir
    
    def get_orders(self):
        '''
            get order context information
        '''
        orders = pd.read_csv(self.raw_data_dir + 'orders.csv')[:50000]
        orders = orders.fillna(0.0)
        orders['days'] = orders.groupby(['user_id'])['days_since_prior_order'].cumsum()
        orders['days_last'] = orders.groupby(['user_id'])['days'].transform(max)
        orders['days_up_to_last'] = orders['days_last'] - orders['days']
        del orders['days_last']
        del orders['days']
        return orders
    
    def get_orders_items(self, prior_or_train):
        '''
            get detailed information of prior or train orders 
        '''
        orders_products = pd.read_csv(self.raw_data_dir + 'order_products__%s.csv'%prior_or_train)
        return orders_products
    
    def get_users_orders(self, prior_or_train):
        '''
            get users' prior detailed orders
        '''
        if os.path.exists(self.cache_dir + 'users_orders.pkl'):
            with open(self.cache_dir + 'users_orders.pkl', 'rb') as f:
                users_orders = pickle.load(f)
        else:
            orders = self.get_orders()
            order_products_prior = self.get_orders_items(prior_or_train)
            users_orders = pd.merge(order_products_prior, orders[['user_id', 'order_id', 'order_number', 'days_up_to_last']], 
                        on = ['order_id'], how = 'left')
            with open(self.cache_dir + 'users_orders.pkl', 'wb') as f:
                pickle.dump(users_orders, f, pickle.HIGHEST_PROTOCOL)
        return users_orders
    
    def get_users_products(self, prior_or_train):
        '''
            get users' all purchased products
        '''
        if os.path.exists(self.cache_dir + 'users_products.pkl'):
            with open(self.cache_dir + 'users_products.pkl', 'rb') as f:
                users_products = pickle.load(f)
        else:
            users_products = self.get_users_orders(prior_or_train)[['user_id', 'product_id']].drop_duplicates()
            users_products['product_id'] = users_products.product_id.astype(int)
            users_products['user_id'] = users_products.user_id.astype(int)
            users_products = users_products.groupby(['user_id'])['product_id'].apply(list).reset_index()
            with open(self.cache_dir + 'users_products.pkl', 'wb') as f:
                pickle.dump(users_products, f, pickle.HIGHEST_PROTOCOL)
        return users_products

    def get_items(self, gran):
        '''
            get items' information
            gran = [departments, aisles, products]
        '''
        items = pd.read_csv(self.raw_data_dir + '%s.csv'%gran)
        return items
    
    def get_baskets(self, prior_or_train, reconstruct = False, none_idx = 49689):
        '''
            get users' baskets
        '''
        filepath = self.cache_dir + './basket_' + prior_or_train + '.pkl'
       
        if os.path.exists(filepath):
            with open(filepath, 'rb') as f:
                up_basket = pickle.load(f)
        else:          
            up = self.get_users_orders(prior_or_train).sort_values(['user_id', 'order_number', 'product_id'], ascending = True)
            uid_oid = up[['user_id', 'order_number']].drop_duplicates()
            up = up[['user_id', 'order_number', 'product_id']]
            up_basket = up.groupby(['user_id', 'order_number'])['product_id'].apply(list).reset_index()
            up_basket = pd.merge(uid_oid, up_basket, on = ['user_id', 'order_number'], how = 'left')
            for row in up_basket.loc[up_basket.product_id.isnull(), 'product_id'].index:
                up_basket.at[row, 'product_id'] = [none_idx]
            up_basket = up_basket.sort_values(['user_id', 'order_number'], ascending = True).groupby(['user_id'])['product_id'].apply(list).reset_index()
            up_basket.columns = ['user_id', 'basket']
            with open(filepath, 'wb') as f:
                pickle.dump(up_basket, f, pickle.HIGHEST_PROTOCOL)
        return up_basket
        
    def get_item_history(self, prior_or_train, reconstruct = False, none_idx = 49689):
        filepath = self.cache_dir + './item_history_' + prior_or_train + '.pkl'
        if (not reconstruct) and os.path.exists(filepath):
            with open(filepath, 'rb') as f:
                item_history = pickle.load(f)
        else:
            up = self.get_users_orders(prior_or_train).sort_values(['user_id', 'order_number', 'product_id'], ascending = True)
            item_history = up.groupby(['user_id', 'order_number'])['product_id'].apply(list).reset_index()
            item_history.loc[item_history.order_number == 1, 'product_id'] = item_history.loc[item_history.order_number == 1, 'product_id'] + [none_idx]
            item_history = item_history.sort_values(['user_id', 'order_number'], ascending = True)
            # accumulate 
            item_history['product_id'] = item_history.groupby(['user_id'])['product_id'].transform(pd.Series.cumsum)
            # get unique item list
            item_history['product_id'] = item_history['product_id'].apply(set).apply(list)
            item_history = item_history.sort_values(['user_id', 'order_number'], ascending = True)
            # shift each group to make it history
            item_history['product_id'] = item_history.groupby(['user_id'])['product_id'].shift(1)
            for row in item_history.loc[item_history.product_id.isnull(), 'product_id'].index:
                item_history.at[row, 'product_id'] = [none_idx]
            item_history = item_history.sort_values(['user_id', 'order_number'], ascending = True).groupby(['user_id'])['product_id'].apply(list).reset_index()
            item_history.columns = ['user_id', 'history_items']

            with open(filepath, 'wb') as f:
                pickle.dump(item_history, f, pickle.HIGHEST_PROTOCOL)
        return item_history 


class TaFengBasketConstructor(object):
    def __init__(self):
        pass
    
    def get_baskets(self):
        with open('./data/TaFeng/user_tran', 'r') as fd:
            lines = [l.strip().split()[2:-1] for l in fd.readlines()]
        with open('./data/TaFeng/user_tran', 'r') as fd:
            user_ids = [l.strip().split()[0] for l in fd.readlines()]
        with open('./data/TaFeng/user_tran', 'r') as fd:
            transaction_id = [l.strip().split()[1] for l in fd.readlines()]

        df = pd.DataFrame()
        df['User Id'] = user_ids
        df['Transaction Id'] = transaction_id
        df['Products'] = lines

        df = df.groupby(['User Id', 'Transaction Id']).agg(sum).reset_index()
        
        all_baskets = []
        for user in df['User Id']:
            all_baskets.append([])
            df_tmp = df[df['User Id'] == user]
            for trans in df_tmp['Transaction Id']:
                all_baskets[-1].append(df_tmp[df_tmp['Transaction Id'] == trans]['Products'].values[0])
        
        return all_baskets

In [6]:
class EmbeddingWrapper(object):
    def __init__(self, type):
        if type == 'product':
            # self.model = Word2Vec.load("product2vec_min_count_50.model")
            self.model = Word2Vec.load("product2vec.model")
            
        elif type == 'aisles':
            self.model = Word2Vec.load("aisles2vec_min_count_50.model")
            self.products = pd.read_csv("./Dataset/instacart-market-basket-analysis/products.csv")
            self.p2aisles = dict(zip(self.products.product_id.astype(str), self.products.aisle_id.astype(str)))
            
        elif type == 'tafeng_products':
            self.model = Word2Vec.load("tafeng2vec_min_count_50.model")
            
        self.vocab_len = len(self.model.wv)
        self.word2index = dict(zip([self.model.wv.index_to_key[i] for i in range(self.vocab_len)],
                              [i for i in range(self.vocab_len)]))
        self.word_index_df = pd.DataFrame(data=list(self.word2index.items()), columns=['product_id', 'emb_id'])
        
    def p2aisle_f(self, i):
        return self.p2aisles[i]

    def lookup_ind_f(self, i):
        return self.word2index[i]

    def get_closest_of_set(self, item_id, set_of_candidates):
        vec_of_interest = self.model.wv.vectors[item_id]
        closest = np.argmin([euclidean(vec_of_interest, self.model.wv.vectors[x]) for x in set_of_candidates])
        return set_of_candidates[closest]
    
    def find_closest_from_preds(self, pred, candidates_l_l):
        closest_from_history = []
        for p in pred:
            closest_from_history.append(self.get_closest_of_set(p, [x for seq in candidates_l_l for x in seq]))
        return closest_from_history
        
    def basket_dist_REMD(self, baskets):
        #Relaxed EMD as lower bound. It is basically a nearest neighborhood search to 
        #find the closest word in doc B for each word in doc A and then take sum of all minimum distances.    
        basket1_vecs = self.model.wv.vectors[[x for x in baskets[0]]]
        basket2_vecs = self.model.wv.vectors[[x for x in baskets[1]]]
        
        distance_matrix = cdist(basket1_vecs, basket2_vecs)
        
        return max(np.mean(np.min(distance_matrix, axis=0)),
                   np.mean(np.min(distance_matrix, axis=1)))
        
    def basket_dist_EMD(self, baskets):
        basket1 = baskets[0]
        basket2 = baskets[1]
        dictionary = np.unique(list(basket1) + list(basket2))
        vocab_len_ = len(dictionary)
        product2ind = dict(zip(dictionary, np.arange(vocab_len_)))

        # Compute distance matrix.
        dictionary_vecs = self.model.wv.vectors[[x for x in dictionary]]
        distance_matrix = squareform(pdist(dictionary_vecs))

        if np.sum(distance_matrix) == 0.0:
            # `emd` gets stuck if the distance matrix contains only zeros.
            return float('inf')

        def nbow(document):
            bow = np.zeros(vocab_len_, dtype=np.float32)
            for d in document:
                bow[product2ind[d]] += 1.
            return bow / len(document)

        # Compute nBOW representation of documents.
        d1 = nbow(basket1)
        d2 = nbow(basket2)

        # Compute WMD.
        return ot.emd2(d1, d2, distance_matrix)
        
    def remove_products_wo_embeddings(self, all_baskets):
        all_baskets_filtered = []
        for s in all_baskets:
            s_cp = []
            for b in s:
                b_cp = [x for x in b if x in self.model.wv.index_to_key]
                if len(b_cp) > 0:
                    s_cp.append(b_cp)
            if len(s_cp) > 0:
                all_baskets_filtered.append(s_cp)
        return all_baskets_filtered

In [7]:
def nested_change(item, func):
    if isinstance(item, list):
        return [nested_change(x, func) for x in item]
    return func(item)


def remove_products_which_are_uncommon(all_baskets, max_num=500):
    print('Removing all but {} most common products'.format(max_num))
    p = []
    for s in all_baskets:
        for b in s:
            p.extend(b)
    product_counter = Counter(p)
    most_common_products = [x for x, _ in product_counter.most_common(max_num)]
    all_baskets_filtered = []
    for s in all_baskets:
        s_cp = []
        for b in s:
            b_cp = [x for x in b if x in most_common_products]
            if len(b_cp) > 0:
                s_cp.append(b_cp)
        if len(s_cp) > 0:
            all_baskets_filtered.append(s_cp)
    return all_baskets_filtered


def remove_short_baskets(all_baskets, l_b = 5, l_s = 10):
    all_baskets_filtered = []
    for s in all_baskets:
        s_cp = []
        for b in s:
            if len(b) > l_b:
                s_cp.append(b)
        if len(s_cp) > l_s:
            all_baskets_filtered.append(s_cp)
    return all_baskets_filtered


def split_data(all_baskets):
    train_ub, test_ub = train_test_split(all_baskets, test_size=0.05, random_state=0)
    train_ub, val_ub = train_test_split(train_ub, test_size=0.05, random_state=0)
    
    test_ub_input = [x[:-1] for x in test_ub]
    test_ub_target = [x[-1] for x in test_ub]
    
    val_ub_input = [x[:-1] for x in val_ub]
    val_ub_target = [x[-1] for x in val_ub]
    
    return train_ub, val_ub_input, val_ub_target, test_ub_input, test_ub_target

In [8]:
class KnnDtw(object):    
    def __init__(self, n_neighbors):
        self.n_neighbors = n_neighbors  
        self.length_to_consider = 10
    
    def _spring_dtw_distance(self, ts_a, ts_b, best_for_ts_a, d, d_lower_bound):
        """Returns the DTW subsequence similarity distance between two 2-D
        timeseries numpy arrays.
        
        Following Subsequence Matching in Data Streams, Machiko Toyoda, Yasushi Sakurai

        Arguments
        ---------
        ts_a, ts_b : array of shape [n_samples, n_timepoints]
            Two arrays containing n_samples of timeseries data
            whose DTW distance between each sample of A and B
            will be compared
            
        best_for_ts_a: list of length n_neighbors. The entries denote the
            stortest distances found so far. This is for stopping the 
            calculation early utilizing a lower bound approximation.
        
        d : DistanceMetric object the distance measure used for market baskets A_i - B_j 
        in the DTW dynamic programming function
        
        d_lower_bound : Lower bound of DistanceMetric object the distance measure used for market baskets A_i - B_j 
        in the DTW dynamic programming function
        
        Returns
        -------
        DTW distance between A and B
        """
        
        # Create cost matrix via broadcasting with large int
        M, N = len(ts_a), len(ts_b)

        #Compute REMD distances
        REMD_gen = map(d_lower_bound, [(i,j) for i in ts_a for j in ts_b])
        d_REMD_min = np.fromiter(REMD_gen, dtype=np.float64)

        #Break here if there is no chance that this is the shortest
        if np.sum(d_REMD_min[np.argpartition(d_REMD_min, M)][:M]) > max(best_for_ts_a):
            return np.inf, ts_b[0]

        cost = np.inf * np.ones((M, N))

        #Compute all distances
        d_mat = np.zeros((M,N))
        for i in range(M):
            for j in range(N):
                d_mat[i,j] = d((ts_a[i], ts_b[j]))

        # Initialize the first row and column
        cost[0, 0] = d((ts_a[0], ts_b[0]))
        for i in range(1, M):
            cost[i, 0] = cost[i-1, 0] + d_mat[i, 0]

        for j in range(1, N):
            cost[0, j] = d_mat[0, j]
            
        # Populate rest of cost matrix within window
        for i in range(1, M):
            w = 1.
            for j in range(1, N):
                choices = cost[i-1, j-1], cost[i, j-1], cost[i-1, j]
                cost[i, j] = min(choices) + w * d_mat[i,j]

        min_idx = np.argmin(cost[-1,:-1])
        # Return DTW distance, prediction for next basket
        return cost[-1,min_idx], ts_b[min_idx + 1]
  
    def _dist_matrix(self, x, y, d, d_lower_bound):
        # Compute full distance matrix of dtw distnces between x and y
        # x_s = np.shape(x)
        # y_s = np.shape(y)
        x_s = [len(x)]
        y_s = [len(y)]
        dm = np.inf * np.ones((x_s[0], y_s[0])) 
        next_baskets = np.empty((x_s[0], y_s[0]), dtype=object)
        
        for i in tqdm(range(0, x_s[0])):
            # Ensure all elements of x have the same length
            max_length = max(len(seq) for seq in x[i])
            x[i] = [np.array(seq)[-max_length:] for seq in x[i]]

            best_dist = [np.inf] * max(self.n_neighbors)
            for j in range(0, y_s[0]):
                # Ensure all elements of y have the same length
                max_length_y = max(len(seq_y) for seq_y in y[j])
                y[j] = [np.array(seq_y)[-max_length_y:] for seq_y in y[j]]

                dist, pred = self._spring_dtw_distance(x[i], y[j], best_dist, d, d_lower_bound)
                if dist < np.max(best_dist):
                    best_dist[np.argmax(best_dist)] = dist               
                dm[i, j] = dist
                next_baskets[i, j] = pred
    
        return dm, next_baskets
        
    def predict(self, tr_d, te_d, d, d_lower_bound):
        dm, predictions = self._dist_matrix(te_d, tr_d, d, d_lower_bound)
        
        preds_total_l = []
        distances_total_l = []
        for k in self.n_neighbors:
            # Identify the k nearest neighbors
            knn_idx = dm.argsort()[:, :k]
            preds_k_l = []
            distances_k_l = []
                
            for i in range(len(te_d)):
                preds = [predictions[i][knn_idx[i][x]] for x in range(knn_idx.shape[1])]
                distances = np.mean([dm[i][knn_idx[i][x]] for x in range(knn_idx.shape[1])])
                pred_len = int(np.mean([len(te_d[i][x]) for x in range(len(te_d[i]))]))
                preds = [x for x, y in Counter([n for s in preds for n in s]).most_common(pred_len)]                
                preds_k_l.append(preds)
                distances_k_l.append(distances)
            preds_total_l.append(preds_k_l)
            distances_total_l.append(distances_k_l)
            
        return preds_total_l, distances_total_l

In [9]:
def run():
    embedding_wrapper = EmbeddingWrapper('product')
    bc = BasketConstructor('./Dataset/instacart-market-basket-analysis/', './Dataset/instacart-market-basket-analysis/')
    ub_basket = bc.get_baskets('prior', reconstruct=False)

    all_baskets = ub_basket.basket.values
    all_baskets = nested_change(list(all_baskets), str)
    print(all_baskets)

    all_baskets = embedding_wrapper.remove_products_wo_embeddings(all_baskets)
    all_baskets = remove_products_which_are_uncommon(all_baskets)
    all_baskets = remove_short_baskets(all_baskets)
    all_baskets = nested_change(all_baskets, embedding_wrapper.lookup_ind_f)

    train_ub, val_ub_input, val_ub_target, test_ub_input, test_ub_target = split_data(all_baskets)

    knndtw = KnnDtw(n_neighbors=[5])
    print(train_ub)
    preds_all, distances = knndtw.predict(train_ub, val_ub_input, embedding_wrapper.basket_dist_EMD, 
                                          embedding_wrapper.basket_dist_REMD)
    return preds_all, distances
    

if __name__ == "__main__":
    run()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Removing all but 500 most common products
[[[67, 357, 120, 73, 77, 257, 15], [290, 7, 339, 164, 142, 73, 121, 130, 92, 71, 6], [273, 454, 144, 2, 469, 3, 339, 142, 73, 92, 45, 6], [273, 2, 246, 469, 120, 0, 142, 57, 73, 77, 121, 130, 257, 92, 218, 5], [273, 225, 333, 67, 454, 144, 469, 3, 339, 142, 57, 73, 227, 6], [67, 30, 327, 0, 142, 57, 73, 381, 92, 227, 6], [67, 246, 469, 164, 0, 57, 73, 121, 92, 227, 5], [67, 368, 234, 246, 469, 11, 327, 12, 57, 73, 9, 284, 201, 203, 92, 6], [67, 239, 469, 201, 92, 227, 6], [27, 246, 469, 0, 73, 9, 6, 5], [273, 27, 20, 67, 368, 469, 164, 0, 57, 73, 9, 130, 33, 6, 5], [225, 363, 246, 0, 73, 161, 14, 247], [368, 118, 246, 0, 73, 9, 121, 315, 5, 15], [225, 36, 368, 254, 164, 12, 57, 121, 33, 238, 218, 315, 227, 6, 5, 15], [273, 67, 363, 368, 246, 0, 73, 9, 130, 6, 5, 15], [27, 263, 225, 67, 368, 246, 3, 339, 73, 201, 33, 53, 6, 5, 15], [20, 67, 36, 368, 246, 254, 0, 9, 65, 161, 201, 238, 5, 15], [273, 263, 225, 368, 2, 246, 254, 339, 164, 0, 12, 9, 

100%|███████████████████████████████████████████| 19/19 [05:59<00:00, 18.93s/it]


In [10]:
pip list

Package                       Version
----------------------------- ------------
aiobotocore                   2.5.0
aiofiles                      22.1.0
aiohttp                       3.8.5
aioitertools                  0.7.1
aiosignal                     1.2.0
aiosqlite                     0.18.0
alabaster                     0.7.12
anaconda-anon-usage           0.4.2
anaconda-catalogs             0.2.0
anaconda-client               1.12.1
anaconda-cloud-auth           0.1.3
anaconda-navigator            2.5.2
anaconda-project              0.11.1
anyio                         3.5.0
appdirs                       1.4.4
applaunchservices             0.3.0
appnope                       0.1.2
appscript                     1.1.2
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.3
astroid                       2.14.2
astropy                       5.1
asttokens                     2.0.5
async-timeout                 4.0.2
atomicwrites 

Note: you may need to restart the kernel to use updated packages.
